In [83]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [84]:
route_movies = "Datasets/Netflix/Netflix_Dataset_Movie.csv"
route_ratings = "Datasets/Netflix/Netflix_Dataset_Rating.csv"

In [85]:
df_movie = pd.read_csv(route_movies)
df_rating = pd.read_csv(route_ratings)
#df_users = pd.read_csv(route_users)


In [86]:
df_movie.head(10)

,Movie_ID,Year,Name
0,1,2003,Dinosaur Planet
1,2,2004,Isle of Man TT 2004 Review
2,3,1997,Character
3,4,1994,Paula Abdul's Get Up & Dance
4,5,2004,The Rise and Fall of ECW
5,6,1997,Sick
6,7,1992,8 Man
7,8,2004,What the #$*! Do We Know!?
8,9,1991,Class of Nuke 'Em High 2
9,10,2001,Fighter


In [87]:
df_rating.head(10)

,User_ID,Rating,Movie_ID
0,712664,5,3
1,1331154,4,3
2,2632461,3,3
3,44937,5,3
4,656399,4,3
5,439011,1,3
6,1644750,3,3
7,2031561,4,3
8,616720,4,3
9,2467008,4,3


In [88]:
df_rating.loc[df_rating["Movie_ID"]==1]

,User_ID,Rating,Movie_ID


In [89]:
total_data = df_rating

In [90]:
total_data = total_data.merge(df_movie,on="Movie_ID")

In [91]:
total_data.head()

,User_ID,Rating,Movie_ID,Year,Name
0,712664,5,3,1997,Character
1,1331154,4,3,1997,Character
2,2632461,3,3,1997,Character
3,44937,5,3,1997,Character
4,656399,4,3,1997,Character


In [92]:
total_data.isnull().sum()

User_ID     0
Rating      0
Movie_ID    0
Year        0
Name        0
dtype: int64

In [93]:
type(total_data["Movie_ID"][0])

numpy.int64

In [94]:
type(total_data["Rating"][0])

numpy.int64

In [95]:
total_data.loc[total_data["Movie_ID"]==1]

,User_ID,Rating,Movie_ID,Year,Name


In [96]:
df_movie.isnull().sum()

Movie_ID    0
Year        0
Name        0
dtype: int64

In [97]:
df_rating.isnull().sum()

User_ID     0
Rating      0
Movie_ID    0
dtype: int64

<h4>Training</h4>

We only need to train algorithm User ID, Movie ID and rating. 

In [98]:
unique_users = np.unique(df_rating["User_ID"])
len(unique_users)

143458

In [99]:
unique_movies = np.unique(df_rating["Movie_ID"])
len(unique_movies)

1350

In [100]:
df_movie["Movie_ID"].nunique()

17770

In [101]:
np.unique(df_rating["User_ID"])

array([      6,       7,      79, ..., 2649388, 2649426, 2649429],
      dtype=int64)

In [102]:
np.unique(df_rating["Movie_ID"])

array([   3,    8,   16, ..., 4492, 4493, 4496], dtype=int64)

In [103]:
def sparse_matrix(dataframe):
    nUsr_unq = dataframe["User_ID"].nunique()
    nMov_unq = dataframe["Movie_ID"].nunique()
    usr_map = dict(zip(np.unique(dataframe["User_ID"]),list(range(nUsr_unq))))
    mov_map = dict(zip(np.unique(dataframe["Movie_ID"]),list(range(nMov_unq))))
    
    usr_inver_map = dict(zip(list(range(nUsr_unq)),np.unique(dataframe["User_ID"])))
    mov_inver_map = dict(zip(list(range(nMov_unq)),np.unique(dataframe["Movie_ID"])))
    
    usr_indx = [usr_map[i] for i in dataframe["User_ID"]]
    mov_indx = [mov_map[i] for i in dataframe["Movie_ID"]]
    sparse_mx = csr_matrix((dataframe["Rating"],(mov_indx,usr_indx)),shape=(nMov_unq,nUsr_unq))
    
    return sparse_mx,usr_map,mov_map,usr_inver_map,mov_inver_map

In [104]:
sparse_mx,usr_map,mov_map,usr_inver_map,mov_inver_map = sparse_matrix(df_rating)

In [105]:
sparse_mx.shape

(1350, 143458)

In [106]:
print(sparse_mx)

  (0, 65)	4
  (0, 185)	4
  (0, 241)	4
  (0, 319)	5
  (0, 337)	4
  (0, 518)	4
  (0, 742)	5
  (0, 824)	2
  (0, 878)	4
  (0, 882)	4
  (0, 1177)	3
  (0, 1182)	4
  (0, 1231)	4
  (0, 1312)	4
  (0, 1351)	4
  (0, 1467)	5
  (0, 1559)	4
  (0, 1609)	3
  (0, 1767)	2
  (0, 1784)	4
  (0, 2049)	2
  (0, 2058)	5
  (0, 2174)	4
  (0, 2249)	5
  (0, 2456)	3
  :	:
  (1349, 143014)	3
  (1349, 143025)	4
  (1349, 143036)	4
  (1349, 143050)	2
  (1349, 143079)	3
  (1349, 143100)	4
  (1349, 143101)	3
  (1349, 143120)	3
  (1349, 143177)	5
  (1349, 143185)	4
  (1349, 143217)	5
  (1349, 143254)	5
  (1349, 143258)	5
  (1349, 143261)	4
  (1349, 143279)	3
  (1349, 143300)	1
  (1349, 143308)	4
  (1349, 143332)	5
  (1349, 143350)	5
  (1349, 143368)	4
  (1349, 143376)	4
  (1349, 143401)	3
  (1349, 143408)	5
  (1349, 143422)	5
  (1349, 143448)	4


In [108]:
def knn_recomendation(movieID,sparse_mx,mov_map,k,metric="cosine",show_distance=False):
    #k=Numero de recomendaciones
    #print(type(movieID))
    neighboursID = []
    mov_ind = mov_map[movieID]
    #print(f"tipo de dato mov_ind {type(mov_ind)}")
    #print(f"dato mov_ind {mov_ind}")
    mov_vector = sparse_mx[mov_ind]
    k+=1
    knn = NearestNeighbors(n_neighbors=k,algorithm="brute",metric=metric)
    knn.fit(sparse_mx)
    if isinstance(mov_vector,(np.ndarray)):
        mov_vector = mov_vector.reshape(1,-1)
    neighbour = knn.kneighbors(mov_vector,return_distance=show_distance)
    for i in range(0,k):
        n=neighbour.item(i)
        neighboursID.append(mov_inver_map[n])
    neighboursID.pop(0)
    return neighboursID

In [109]:
movies_titles = dict(zip(df_movie["Movie_ID"],df_movie["Name"]))
movie_id = 3
print(f'Pelicula elegida= {movies_titles[movie_id]}')

Pelicula elegida= Character


In [110]:
#movies_titles

In [111]:
movies_recomendation_final = knn_recomendation(movie_id,sparse_mx,mov_map,k=10)
movieTitle = movies_titles[movie_id]
print(f" Pelicula vista originalmente :::::::::>>>>>> {movieTitle}")
print("***************************************************************************")
print("Peliculas Recomendadas")
print("***************************************************************************")
for i in movies_recomendation_final:
    print(movies_titles[i])

 Pelicula vista originalmente :::::::::>>>>>> Character
***************************************************************************
Peliculas Recomendadas
***************************************************************************
Burnt by the Sun
Pelle the Conqueror
Man on the Train
Talk to Her
Indochine
East/West
Mostly Martha
Il Postino
With a Friend Like Harry
Live Flesh
